In [17]:
import sys; sys.path.append("../../../../automl/")

In [18]:
from pathlib import Path
import yaml
import joblib

import pandas as pd
import numpy as np
import optuna

from sklearn.model_selection import train_test_split, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score

# from src.automl.model.lama import TabularLama
# from src.automl.loggers import configure_root_logger
# from src.automl.constants import create_ml_data_dir
# from src.automl.model.metrics import RocAuc

In [19]:
optuna.logging.set_verbosity(optuna.logging.INFO)

In [20]:
# create_ml_data_dir()
# configure_root_logger()

## Constants

In [21]:
RANDOM_SEED = 77
DATA_PATH = Path("../../../../data/")
CONFIG_PATH = Path("../../../../configs/config.yaml")
N_JOBS = 16

In [22]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

## Data

In [23]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc_oof.parquet")
#df_train, df_test = df_train.sort_values(by="id").iloc[:300_000], df_train.sort_values(by="id").iloc[300_000:]

In [24]:
df_train["target"].value_counts(normalize=True)

0    0.940982
1    0.059018
Name: target, dtype: float64

In [25]:
# undersample the 0 class
#df_train = pd.concat([df_train.loc[df_train.target == 1], df_train.loc[df_train.target == 0].sample(200_000, random_state=RANDOM_SEED)], ignore_index=True)


In [26]:
cat_columns = df_train.drop(columns=["target", "id"]).select_dtypes(int).columns.values.tolist()

In [27]:
X_train, y_train = df_train[cfg["selected_features"] + cfg["stack_features"] + cat_columns], df_train["target"]
#X_test, y_test = df_test[cfg["selected_features"] + cat_columns], df_test["target"]

In [28]:
display(y_train.value_counts(normalize=True))
#display(y_test.value_counts(normalize=True))

0    0.940982
1    0.059018
Name: target, dtype: float64

In [29]:
#categorical_features = ohe_cols# + oe_cols

## Blend

In [30]:
cfg["stack_features"]

['lamau_81425_full_dataset',
 'lgb_8122_full_dataset',
 'cb_8114_full_dataset',
 'xgb_81325_full_dataset',
 'lama_81298_full_dataset',
 'lamann_autoint_8053_full_dataset',
 'lamann_fttransformer_8050_full_dataset']

In [57]:
df_train[cfg["stack_features"]] = df_train[cfg["stack_features"]].astype(np.float32)

In [59]:
cfg["stack_features"] = ["lamau_81425_full_dataset", "xgb_81325_full_dataset", "lamann_autoint_8053_full_dataset"]

In [17]:
def objective(trial):
    weights = []
    w0 = trial.suggest_float("w0", 0, 1)
    weights.append(w0)
    
    for i in range(1, len(cfg["stack_features"]) - 1):
        sum_prev_weights = np.sum(weights)
        wi = trial.suggest_float(f"w{i}", 0, sum_prev_weights)
        weights.append(wi)
        
    w6 = 1 - np.sum(weights)
    weights.append(w6)
    
    weights = np.array(weights)
    
    cv = kf.split(X_train, y_train)
    
    metrics = []
    for i, (train_idx, test_idx) in enumerate(cv):
    
        preds = np.sum(X_train.iloc[test_idx][cfg["stack_features"]].to_numpy() * weights, axis=1)
        metrics.append(metric(y_train.iloc[test_idx], preds))
        
    
    return np.mean(metrics)
    
    

In [15]:
def objective(trial):
    weights = []
    w0 = trial.suggest_float("w0", 0, 1)
    weights.append(w0)
    
    
        
    w1 = 1 - w0
    weights.append(w1)
    
    weights = np.array(weights)
    
    cv = kf.split(X_train, y_train)
    
    metrics = []
    for i, (train_idx, test_idx) in enumerate(cv):
    
        preds = np.sum(X_train.iloc[test_idx][cfg["stack_features"]].to_numpy() * weights, axis=1)
        metrics.append(metric(y_train.iloc[test_idx], preds))
        
    
    return np.mean(metrics)
    

In [60]:
def objective(trial):
    weights = []
    w0 = trial.suggest_float("w0", 0, 1)
    weights.append(w0)
    
    w1 = trial.suggest_float("w1", 0, 1 - w0)
    weights.append(w1)
        
    w2 = 1 - np.sum(weights)
    weights.append(w2)
    
    weights = np.array(weights)
    
    preds = np.sum(X_train[cfg["stack_features"]].to_numpy() * weights, axis=1)
    
    res = metric(y_train, preds)
        
    
    return res
    

In [61]:
# kf = StratifiedKFold(
#                 n_splits=5, random_state=RANDOM_SEED, shuffle=True)

# kf = TimeSeriesSplit(
#                 n_splits=5)
metric = roc_auc_score

In [62]:
study = optuna.create_study(
            study_name="blending",
            direction="maximize")
study.optimize(objective, timeout=60, n_jobs=N_JOBS)

[I 2024-11-10 15:15:20,855] A new study created in memory with name: blending
[I 2024-11-10 15:15:21,111] Trial 1 finished with value: 0.8142948610934067 and parameters: {'w0': 0.8083097182761826, 'w1': 0.09585785296412204}. Best is trial 1 with value: 0.8142948610934067.
[I 2024-11-10 15:15:21,124] Trial 2 finished with value: 0.8140855680613288 and parameters: {'w0': 0.5039870676349768, 'w1': 0.32835758089828276}. Best is trial 1 with value: 0.8142948610934067.
[I 2024-11-10 15:15:21,125] Trial 4 finished with value: 0.8137217738065184 and parameters: {'w0': 0.19361394737868876, 'w1': 0.6636660716291197}. Best is trial 1 with value: 0.8142948610934067.
[I 2024-11-10 15:15:21,148] Trial 5 finished with value: 0.8111232963453153 and parameters: {'w0': 0.3591805998750405, 'w1': 0.028622613031281482}. Best is trial 1 with value: 0.8142948610934067.
[I 2024-11-10 15:15:21,157] Trial 0 finished with value: 0.8138438606527039 and parameters: {'w0': 0.24166382200765169, 'w1': 0.7070439532682

[I 2024-11-10 15:15:21,976] Trial 44 finished with value: 0.8143103957067811 and parameters: {'w0': 0.9157888056090918, 'w1': 0.05193743025802988}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:22,031] Trial 47 finished with value: 0.8143140557925155 and parameters: {'w0': 0.8901764380056634, 'w1': 0.059537368814477645}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:22,040] Trial 46 finished with value: 0.8143156146222179 and parameters: {'w0': 0.9040727223618973, 'w1': 0.06012240967026369}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:22,052] Trial 45 finished with value: 0.8143106311132692 and parameters: {'w0': 0.8853111177546837, 'w1': 0.057212682994390245}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:22,073] Trial 40 finished with value: 0.8136541498123144 and parameters: {'w0': 0.5818578975787192, 'w1': 0.12459385875045689}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10

[I 2024-11-10 15:15:23,012] Trial 84 finished with value: 0.8142540938998142 and parameters: {'w0': 0.7668674281262933, 'w1': 0.10940075000622915}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:23,039] Trial 85 finished with value: 0.8142240748264717 and parameters: {'w0': 0.7559729204329554, 'w1': 0.10583119263551498}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:23,082] Trial 88 finished with value: 0.8142565830703015 and parameters: {'w0': 0.7751035401405042, 'w1': 0.10321397950857993}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:23,084] Trial 87 finished with value: 0.8142360467018662 and parameters: {'w0': 0.7641601168231937, 'w1': 0.10343274775179334}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:23,120] Trial 89 finished with value: 0.8142397944322207 and parameters: {'w0': 0.768197272932921, 'w1': 0.10152618318819742}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15

[I 2024-11-10 15:15:24,082] Trial 126 finished with value: 0.8142293889802861 and parameters: {'w0': 0.8016386803362235, 'w1': 0.06766045124311433}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:24,098] Trial 127 finished with value: 0.8143194268995126 and parameters: {'w0': 0.8697137007533374, 'w1': 0.07439354778137568}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:24,104] Trial 128 finished with value: 0.8143063940074196 and parameters: {'w0': 0.8598050845856541, 'w1': 0.0666738087154886}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:24,127] Trial 129 finished with value: 0.8143078726807906 and parameters: {'w0': 0.8648873941585087, 'w1': 0.06490248977154824}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:24,184] Trial 132 finished with value: 0.8143090812484197 and parameters: {'w0': 0.8665606102306146, 'w1': 0.06501970897591251}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-

[I 2024-11-10 15:15:25,140] Trial 167 finished with value: 0.8104307041953231 and parameters: {'w0': 0.28593131355719714, 'w1': 0.042011963935911785}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:25,144] Trial 168 finished with value: 0.8143261856602575 and parameters: {'w0': 0.8835683665392168, 'w1': 0.0805875868305715}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:25,230] Trial 170 finished with value: 0.8143257327769868 and parameters: {'w0': 0.8841585932975831, 'w1': 0.07955342423300106}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:25,244] Trial 172 finished with value: 0.8143257083082119 and parameters: {'w0': 0.8872505737351697, 'w1': 0.08020246865641703}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-10 15:15:25,288] Trial 171 finished with value: 0.814325345389876 and parameters: {'w0': 0.8890215029520747, 'w1': 0.0794586502806995}. Best is trial 22 with value: 0.8143468406811565.
[I 2024-11-

...

[I 2024-11-10 15:16:13,376] Trial 1589 finished with value: 0.8134662704386221 and parameters: {'w0': 0.4508405841317109, 'w1': 0.22744516906098627}. Best is trial 994 with value: 0.8143545837827411.
[I 2024-11-10 15:16:13,408] Trial 1590 finished with value: 0.8143523082921575 and parameters: {'w0': 0.7822814893232161, 'w1': 0.20257692823837342}. Best is trial 994 with value: 0.8143545837827411.
[I 2024-11-10 15:16:13,423] Trial 1591 finished with value: 0.8143523716789403 and parameters: {'w0': 0.7814675458290968, 'w1': 0.20323386391006484}. Best is trial 994 with value: 0.8143545837827411.
[I 2024-11-10 15:16:13,643] Trial 1596 finished with value: 0.8143519564480515 and parameters: {'w0': 0.78220981013336, 'w1': 0.20397139467998032}. Best is trial 994 with value: 0.8143545837827411.
[I 2024-11-10 15:16:13,655] Trial 1595 finished with value: 0.8143534931292937 and parameters: {'w0': 0.7774018513381623, 'w1': 0.20314428199463902}. Best is trial 994 with value: 0.8143545837827411.
[I

[I 2024-11-10 15:16:15,323] Trial 1633 finished with value: 0.8143535306762067 and parameters: {'w0': 0.7658704855472142, 'w1': 0.19807945424861156}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:15,388] Trial 1635 finished with value: 0.8143541896456234 and parameters: {'w0': 0.7680615964574139, 'w1': 0.19983170159287028}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:15,421] Trial 1639 finished with value: 0.8143523060773115 and parameters: {'w0': 0.7597495511888464, 'w1': 0.19912942237701586}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:15,449] Trial 1636 finished with value: 0.8102219394567486 and parameters: {'w0': 0.0690947502790562, 'w1': 0.2542764382537286}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:15,451] Trial 1642 finished with value: 0.8143541950245351 and parameters: {'w0': 0.7618900802096323, 'w1': 0.20841845857452304}. Best is trial 1598 with value: 0.814354588212433.
[

[I 2024-11-10 15:16:17,038] Trial 1678 finished with value: 0.8143375732381858 and parameters: {'w0': 0.723465059201527, 'w1': 0.21141067129276486}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:17,051] Trial 1676 finished with value: 0.8143394845448071 and parameters: {'w0': 0.726417687606699, 'w1': 0.21069864922347112}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:17,067] Trial 1675 finished with value: 0.8143419307894699 and parameters: {'w0': 0.7230250591309457, 'w1': 0.21902183836814396}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:17,101] Trial 1680 finished with value: 0.8143443202918875 and parameters: {'w0': 0.8241539259611318, 'w1': 0.17405641800373584}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:17,103] Trial 1682 finished with value: 0.8143410006596223 and parameters: {'w0': 0.8399896922854382, 'w1': 0.15888836865483147}. Best is trial 1598 with value: 0.814354588212433.
[I

[I 2024-11-10 15:16:18,670] Trial 1714 finished with value: 0.8143481285613658 and parameters: {'w0': 0.8017107065014245, 'w1': 0.19330590301304812}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:18,711] Trial 1721 finished with value: 0.8143439284750849 and parameters: {'w0': 0.7474923086386078, 'w1': 0.19414837944598576}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:18,732] Trial 1720 finished with value: 0.814347918678341 and parameters: {'w0': 0.8026206047864578, 'w1': 0.19279860549108668}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:18,733] Trial 1717 finished with value: 0.8130887257870864 and parameters: {'w0': 0.38103117414023935, 'w1': 0.23859085572055033}. Best is trial 1598 with value: 0.814354588212433.
[I 2024-11-10 15:16:19,143] Trial 1722 finished with value: 0.8143479066548913 and parameters: {'w0': 0.8032986268907499, 'w1': 0.19201230119905496}. Best is trial 1598 with value: 0.814354588212433.


[I 2024-11-10 15:16:20,583] Trial 1758 finished with value: 0.8143545838882099 and parameters: {'w0': 0.7705844513573294, 'w1': 0.20182049975600413}. Best is trial 1745 with value: 0.8143545901108724.
[I 2024-11-10 15:16:20,660] Trial 1763 finished with value: 0.8143032929066256 and parameters: {'w0': 0.6842537304065716, 'w1': 0.22196406184262887}. Best is trial 1745 with value: 0.8143545901108724.
[I 2024-11-10 15:16:20,672] Trial 1760 finished with value: 0.8143521100107068 and parameters: {'w0': 0.757910459842286, 'w1': 0.200453923306567}. Best is trial 1745 with value: 0.8143545901108724.
[I 2024-11-10 15:16:20,696] Trial 1761 finished with value: 0.8143502576611763 and parameters: {'w0': 0.75464582991772, 'w1': 0.1989315409409188}. Best is trial 1745 with value: 0.8143545901108724.
[I 2024-11-10 15:16:20,714] Trial 1762 finished with value: 0.8139739464679201 and parameters: {'w0': 0.5984237771834644, 'w1': 0.18110299644447828}. Best is trial 1745 with value: 0.8143545901108724.
[

In [63]:
study.best_params

{'w0': 0.7700214315622841, 'w1': 0.201793841670085}

In [64]:
study.best_value

0.8143545901108724

In [72]:
weights = np.array([0.9, 0.08, 0.02])

In [66]:
weights = np.append(weights, 1 - np.sum(weights))

In [37]:
# study = optuna.create_study(
#             study_name="blending",
#             direction="maximize")
# study.optimize(objective, timeout=60, n_jobs=N_JOBS)

In [38]:
# print("w6: ", 1 - np.sum(list(study.best_params.values())))

In [76]:
pred = np.round(np.sum((df_train[cfg["stack_features"]].to_numpy() * weights), axis=1), 10)

In [78]:
metric(df_train["target"], pred)

0.8143227645142177

In [79]:
test = pd.read_parquet(DATA_PATH / "test_preproc_oof.parquet")
#test[cfg["stack_features"]] = test[cfg["stack_features"]].astype(np.float32)
test["target"] = np.round(np.sum((test[cfg["stack_features"]].to_numpy() * weights), axis=1), 10)
test[['id', 'target']].to_csv(f'blend3.csv', index=False)


In [16]:
# res = pd.DataFrame()
# res[MODEL_NAME] = oof[:, 1]
# res.to_csv(MODEL_DIR / "oof.csv", index=False)
# #joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

# with (MODEL_DIR / "params.yaml").open("w") as f:
#     yaml.dump(model.params, f)

# with (MODEL_DIR / "score.txt").open("w") as f:
#     print("OOF:", metric(y_train, oof), file=f)
    
# test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
# test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
# test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

In [24]:
test = pd.read_parquet(DATA_PATH / "test_preproc_oof.parquet")
test[cfg["stack_features"]] = test[cfg["stack_features"]].astype(np.float32)
test["target"] = model.predict(test[cfg["selected_features"] + cfg["stack_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(f'{MODEL_NAME}.csv', index=False)

In [26]:
model.model.get_feature_scores()[:50]

,Feature,Importance
0,lamau_814_full_dataset,78048.337071
1,xgb_81325_full_dataset,29297.001751
2,lama_81298_full_dataset,24098.444188
3,feature_162,1398.695796
4,feature_24,1321.674997
5,feature_18,1298.269506
6,feature_26,1290.120701
7,feature_145,1287.683197
8,feature_78,1279.916003
9,feature_36,1248.758602


In [24]:
imp = pd.DataFrame().assign(names=model.models[0].feature_names_, imp=model.models[0].feature_importances_)

In [41]:
imp.sort_values(by="imp", ascending=False).reset_index(drop=True).query("names == 'feature_185'")

,names,imp
61,feature_185,0.0


In [29]:
cat_columns

['feature_7',
 'feature_31',
 'feature_60',
 'feature_61',
 'feature_71',
 'feature_109',
 'feature_122',
 'feature_156',
 'feature_163',
 'feature_167',
 'feature_179',
 'feature_185']

## With Time series cross val

In [13]:
metric = RocAuc()

In [14]:
df_train = df_train.sort_values(by="id").reset_index(drop=True)
X_train, y_train = df_train[cfg["selected_features"] + cat_columns], df_train["target"]

In [15]:
model = LightGBMClassification(n_jobs=16, time_series=True)
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-07 08:27:33,388] - [   START    ] - Tuning LightGBMClassification
[2024-11-07 08:27:40,118] - [   OPTUNA   ] - Trial 0. New best score 0.7903405446081995 with parameters {'max_depth': 6, 'num_leaves': 488, 'min_data_in_leaf': 188, 'bagging_fraction': 0.7993292420985183, 'bagging_freq': 0, 'feature_fraction': 0.49359671220172163, 'lambda_l1': 0.5808361216819946, 'lambda_l2': 8.661761457749352, 'min_gain_to_split': 12.022300234864176, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:03,043] - [   OPTUNA   ] - Trial 2. New best score 0.7913515589848906 with parameters {'max_depth': 5, 'num_leaves': 194, 'min_data_in_leaf': 117, 'bagging_fraction': 0.8925879806965068, 'bagging_freq': 0, 'feature_fraction': 0.708540663048167, 'lambda_l1': 5.924145688620425, 'lambda_l2': 0.46450412719997725, 'min_gain_to_split': 12.150897038028766, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:12,225] - [   OPTUNA   ] - Trial 3. New best score 0.8048686053278628 with param

In [27]:
none_oofs_idx = oof[np.any(np.isnan(oof), axis=1)].shape[0]

In [32]:
metric(y_train[none_oofs_idx:], oof[none_oofs_idx:])

0.8095227594190041

In [34]:
MODEL_NAME = "lgb_8095_full_dataset_time_series"
MODEL_DIR = Path(f"../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [35]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[none_oofs_idx:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

## TEST 
**81.22112399468679**

## Inference

In [27]:
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv('lgb_813.csv', index=False)

In [25]:
pred_1 = pd.read_csv("lama_utilized.csv")
pred_2 = pd.read_csv("lgmb_oe_ohe_cols_0805.csv")
pred_3 = pd.read_csv("catboost_ts.csv")

In [26]:
pred_1["target"] = 0.6 * pred_1["target"] + 0.2 * pred_2["target"] + 0.2 * pred_3["target"]

In [29]:
pred_1.to_csv("blend.csv", index=False)

In [166]:
MODEL_DIR.open?

Signature:
MODEL_DIR.open(
    mode='r',
    buffering=-1,
    encoding=None,
    errors=None,
    newline=None,
)
Docstring:
Open the file pointed by this path and return a file object, as
the built-in open() function does.
File:      /usr/lib/python3.10/pathlib.py
Type:      method